**《深度学习之 PyTorch 实战》**

讲师作者：[土豆老师](https://iphysresearch.github.io)

# 深度学习计算

- 之前的课程中介绍了包括多层感知机在内的简单深度学习模型的原理和实现。
- 下面我们将简要概括深度学习计算的各个重要组成部分，如模型构造、参数的访问和初始化等，自定义层，读取、存储和使用GPU。
- 通过本章的学习，我们将能够深入了解模型实现和计算的各个细节，并为在之后章节实现更复杂模型打下坚实的基础。

---

## 模型构造

让我们回顾一下在「多层感知机的简洁实现」中含单隐藏层的多层感知机的实现方法。

我们首先构造 `Sequential` 实例，然后依次添加两个全连接层。其中第一层的输出大小为 256，即隐藏层单元个数是 256；第二层的输出大小为10，即输出层单元个数是10。我们在上一章的其他节中也使用了 `Sequential` 类构造模型。这里我们介绍另外一种基于 `Module` 类的模型构造方法：它让模型构造更加灵活。

其实前面我们陆陆续续已经使用了这些方法了，本节系统介绍一下。

### 继承 `Module` 类来构造模型

`Module` 类是 `nn` 模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承 `Module` 类构造本节开头提到的多层感知机。这里定义的 `MLP` 类重载了 `Module` 类的 `__init__` 函数和 `forward` 函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

> 现在我们开始将 `torch.nn` 单独 import 出来。

In [1]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用 MLP 父类 Module 的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数 params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

以上的 `MLP` 类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的 `backward` 函数。

我们可以实例化 `MLP` 类得到模型变量 `net`。下面的代码初始化 `net` 并传入输入数据 `X` 做一次前向计算。其中，`net(X)` 会调用 `MLP` 继承自 `Module` 类的 `__call__` 函数，这个函数将调用 `MLP` 类定义的 `forward` 函数来完成前向计算。

In [2]:
X = torch.rand(2, 784)

net = MLP()
print(net)

net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-1.0750e-02,  6.2731e-02,  2.3870e-01,  1.2594e-02, -1.4003e-02,
          1.7490e-01, -5.6121e-02, -4.4315e-02,  4.6774e-02,  7.7888e-03],
        [ 2.6103e-02,  5.8683e-02,  2.0914e-01, -4.7617e-05,  1.3082e-03,
          1.0819e-01, -5.8243e-02, -1.9335e-01, -7.4059e-02, -7.8000e-02]],
       grad_fn=<AddmmBackward>)

小贴士：
>`__call__` 类方法可以将实例对象变为了可调用对象。

注意，这里并没有将 `Module` 类命名为 Layer（层）或者 Model（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如 PyTorch 提供的 `Linear` 类），又可以是一个模型（如这里定义的 `MLP` 类），或者是模型的一个部分。我们下面通过两个例子来展示它的灵活性。

![](https://vip2.loli.io/2021/03/22/wauQHW8O75xoTjn.png)


### `Module` 的子类

我们刚刚提到，`Module` 类是一个通用的部件。事实上，PyTorch 还实现了继承自 Module 的可以方便构建模型的类: 如 `Sequential`、`ModuleList` 和 `ModuleDict` 等等。


#### `Sequential` 类

当模型的前向计算为简单串联各个层的计算时，`Sequential` 类可以通过更加简单的方式定义模型。这正是 `Sequential` 类的目的：它可以接收一个子模块的有序字典（`OrderedDict`）或者一系列子模块作为参数来逐一添加 `Module` 的实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

下面我们实现一个与 `Sequential` 类有相同功能的 `MySequential` 类。这或许可以帮助读者更加清晰地理解 `Sequential` 类的工作机制。

In [4]:
class MySequential1(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential1, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传入的是一个 OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)  
                # add_module 方法会将 module 添加进 self._modules (一个OrderedDict)
        else:  # 传入的是一些 Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, x):
        # self._modules 返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            x = module(x)
        return x

我们用 `MySequential1` 类来实现前面描述的 `MLP` 类，并使用随机初始化的模型做一次前向计算。

In [5]:
net = MySequential1(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10), 
        )
print(net)
net(X)

MySequential1(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1562, -0.0066, -0.1133, -0.0786,  0.0905,  0.1620, -0.0796,  0.1559,
          0.2265, -0.1160],
        [-0.0826,  0.0676,  0.0339, -0.0423,  0.1989,  0.1573,  0.0096,  0.1382,
          0.1268, -0.0780]], grad_fn=<AddmmBackward>)

可以观察到这里 `MySequential` 类的使用跟「多层感知机的简洁实现」中 `Sequential` 类的使用没什么区别。

我们可以在这里拓展一下！

除了在 `Module` 类中自定义 `__init__` 和 `forward` 类方法实现 `Sequential` 类以外，我们也可以通过直接继承 `Sequential` 类，并使用 `add_module` 类方法来实现模块的串联。

In [6]:
class MySequential2(nn.Sequential):
    def __init__(self, **kwargs):
        super(MySequential2, self).__init__(**kwargs)

    def add_module(self, block):
        # 这里的 block 变量是 Block 子类的实例，并且我们假定其名字是唯一。
        # 在 Block 类中有一个成员变量 _children，它的数据类型就是一个
        # 有序字典 OrderedDict。当我们的 MySequential2 初始化的时候，
        # 程序背后会自动对所有的成员变量进行初始化的。
        self._modules[block] = block

    def forward(self, x):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            x = block(x)
        return x

In [7]:
net = MySequential2()
net.add_module(nn.Linear(784,256))
net.add_module(nn.ReLU())
net.add_module(nn.Linear(256,10))
net(X)

tensor([[ 0.0279, -0.0259,  0.1756,  0.0264, -0.0179, -0.0488,  0.0588, -0.0762,
          0.2578, -0.2208],
        [ 0.0311,  0.0045,  0.0950, -0.1463,  0.1052,  0.1958,  0.0185, -0.2009,
          0.1820, -0.1906]], grad_fn=<AddmmBackward>)

#### `ModuleList` 类

`ModuleList` 接收一个子模块的列表作为输入，然后也可以类似 `List` 那样进行 `append` 和 `extend` 操作:

In [10]:
net = nn.ModuleList([nn.Linear(784, 256), 
                     nn.ReLU()])
net.append(nn.Linear(256, 10)) # 类似 List 的 append 操作
print(net[-1])  # 类似 List 的索引访问
print(net)
# net(X) # 会报 NotImplementedError

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


既然 `Sequential` 和 `ModuleList` 都可以进行列表化构造网络，那二者区别是什么呢。

`ModuleList` 仅仅是一个储存各种模块的列表，这些模块之间没有联系也没有顺序（所以不用保证相邻层的输入输出维度匹配），而且没有实现 `forward` 功能需要自己实现，所以上面执行 `net(X)` 会报 `NotImplementedError`；而 `Sequential` 内的模块需要按照顺序排列，要保证相邻层的输入输出大小相匹配，内部 `forward` 功能已经实现。

`ModuleList` 的出现只是让网络定义前向传播时更加灵活，见下面官网的例子。

In [11]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for _ in range(10)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x

另外，`ModuleList` 不同于一般的 Python 的 list，加入到 `ModuleList` 里面的所有模块的参数会被自动添加到整个网络中，下面看一个例子对比一下。

In [12]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)])

class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        self.linears = [nn.Linear(10, 10)]

net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
for p in net1.parameters():
    print(p.size())

print("net2:")
for p in net2.parameters():
    print(p)

net1:
torch.Size([10, 10])
torch.Size([10])
net2:


#### `ModuleDict` 类

`ModuleDict` 接收一个子模块的字典作为输入, 然后也可以类似字典那样进行添加访问操作:

In [15]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加层
print(net['linear']) # 访问某层
print(net.output)
print(net)
# net(X) # 会报 NotImplementedError

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


和 `ModuleList` 一样，`ModuleDict` 实例仅仅是存放了一些模块的字典，并没有定义 `forward` 函数需要自己定义。同样，`ModuleDict` 也与 Python 的 `Dict` 有所不同，ModuleDict 里的所有模块的参数会被自动添加到整个网络中。

### 构造复杂的模型

虽然上面介绍的这些类可以使模型构造更加简单，且可以不需要定义 `forward` 函数，但直接继承 `Module` 类可以极大地拓展模型构造的灵活性。下面我们构造一个稍微复杂点的网络 `FancyMLP` 。在这个网络中，我们通过 `requires_grad` 创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用 `Tensor` 的函数和 Python 的控制流，并多次调用相同的层。

In [16]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及 torch.nn.functional 中的 relu 函数和 mm 函数
        x = torch.nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用 item 函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

在这个 `FancyMLP` 模型中，我们使用了常数权重 `rand_weight`（注意它不是可训练模型参数）、做了矩阵乘法操作（`torch.mm`）并重复使用了相同的 `Linear` 层。下面我们来测试该模型的前向计算。

In [17]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(4.5269, grad_fn=<SumBackward0>)

因为 `FancyMLP` 和 `Sequential` 类都是 `Module` 类的子类，所以我们可以嵌套调用它们。

In [18]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), 
                                 nn.ReLU()) 

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), 
                    nn.Linear(30, 20), 
                    nn.ReLU())
net.add_module("FancyMLP",FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): ReLU()
  (FancyMLP): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-1.5452, grad_fn=<SumBackward0>)

### 小结

- 可以通过继承 `Module` 类来构造模型。
- `Sequential`、`ModuleList`、`ModuleDict` 类都继承自 `Module` 类。
- 与 `Sequential` 不同，`ModuleList` 和 `ModuleDict` 并没有定义一个完整的网络，它们只是将不同的模块存放在一起，需要自己定义 `forward` 函数。
- 虽然 `Sequential` 等类可以使模型构造更加简单，但直接继承 `Module` 类可以极大地拓展模型构造的灵活性。

---

## 模型参数的访问、初始化和共享

>(Restart your kernel here)

在「线性回归的简洁实现」中，我们通过 `init` 模块来初始化模型的参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。与之前不同的是，在这里我们从 `torch.nn` 中导入了 `init` 模块，它包含了多种模型初始化方法。

> 现在我们开始将 `torch.nn.init` 单独 import 出来。

In [1]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()
Y

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


tensor(-0.4634, grad_fn=<SumBackward0>)

### 访问模型参数

回忆一下上一节中提到的 `Sequential` 类与 `Module` 类的继承关系。对于 `Sequential` 实例中含模型参数的层，我们可以通过 `Module` 类的 `parameters()` 或者 `named_parameters` 方法来访问所有参数（以迭代器的形式返回），后者除了返回参数 `Tensor` 外还会返回其名字。下面，访问多层感知机 `net` 的所有参数：

In [2]:
print(type(net.parameters()))   # parameters()
for param in net.parameters():
    print(param.size())

<class 'generator'>
torch.Size([3, 4])
torch.Size([3])
torch.Size([1, 3])
torch.Size([1])


In [3]:
print(type(net.named_parameters()))  # named_parameters()
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


可见返回的名字自动加上了层数的索引作为前缀。

我们再来访问 `net` 中单层的参数。对于使用 `Sequential` 类构造的神经网络，我们可以通过方括号 `[]` 来访问网络的任一层。索引 `0` 表示隐藏层为 `Sequential` 实例最先添加的层。

In [4]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [7]:
for name, param in net[:3].named_parameters():
    print(name, param.size(), type(param))

0.weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
0.bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>
2.weight torch.Size([1, 3]) <class 'torch.nn.parameter.Parameter'>
2.bias torch.Size([1]) <class 'torch.nn.parameter.Parameter'>


因为这里是单层的所以没有了层数索引的前缀。另外返回的 `param` 的类型为 `torch.nn.parameter.Parameter`，其实这是 `Tensor` 的子类，和 `Tensor` 不同的是如果一个 `Tensor` 是 `Parameter`，那么它会自动被添加到模型的参数列表里，来看下面这个例子。

In [8]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(2, 2))
        self.weight2 = torch.rand(2, 2)
    def forward(self, x):
        pass

net2 = MyModel()
for name, param in net2.named_parameters():
    print(name)

weight1


可以看到，上面的代码中 `weight1` 在参数列表中但是 `weight2` 却没在参数列表中。

因为 `Parameter` 是 `Tensor`，即 `Tensor` 拥有的属性它都有，比如可以根据 `data` 来访问参数数值，用 `grad` 来访问参数梯度。

In [9]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None

Y.backward()
print(weight_0.grad)

tensor([[ 0.4664, -0.3196, -0.0848, -0.2056],
        [-0.1335, -0.0332,  0.2118,  0.2264],
        [ 0.0436,  0.3163, -0.0357,  0.0619]])
None
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0147, -0.0973, -0.0663, -0.1005]])


### 初始化模型参数

我们在「数值稳定性和模型初始化」中提到了 PyTorch 中 `nn.Module` 的模块参数都采取了较为合理的初始化策略（不同类型的 layer 具体采样的哪一种初始化方法的可参考源代码）。但我们经常需要使用其他方法来初始化权重。PyTorch 的 `init` 模块里提供了多种预设的初始化方法。在下面的例子中，我们将权重参数初始化成均值为 0、标准差为 0.01 的正态分布随机数，并依然将偏差参数清零。

In [10]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[-0.0110,  0.0027,  0.0070, -0.0077],
        [ 0.0093, -0.0052,  0.0074, -0.0023],
        [ 0.0040,  0.0012,  0.0077, -0.0097]])
2.weight tensor([[-0.0163, -0.0236,  0.0019]])


下面使用常数来初始化权重参数。

In [11]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 自定义初始化方法

有时候我们需要的初始化方法并没有在 `init` 模块中提供。这时，可以实现一个初始化方法，从而能够像使用其他初始化方法那样使用它。在这之前我们先来看看 PyTorch 是怎么实现这些初始化方法的，例如 `torch.nn.init.normal_`：

```python
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)
```

可以看到这就是一个 inplace 改变 `Tensor` 值的函数，而且这个过程是不记录梯度的。

类似的我们来实现一个自定义的初始化方法。在下面的例子里，我们令权重有一半概率初始化为 0，有另一半概率初始化为 `[−10,−5]` 和 `[5,10]` 两个区间里均匀分布的随机数。

In [12]:
def init_weight_(tensor): # 自定义初始化
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-0.0000, -6.4240, -8.0191, -0.0000],
        [-0.0000, -8.6734,  0.0000, -6.4193],
        [ 9.3927, -0.0000,  9.1377, -0.0000]])
2.weight tensor([[0.0000, 5.9064, -0.0000]])


此外，我们还知道可以通过改变这些参数的 `data` 来改写模型参数值同时不会影响梯度:

In [13]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


### 共享模型参数

在有些情况下，我们希望在多个层之间共享模型参数。在上面讲「创建模型中不被迭代的参数」时提到了如何共享模型参数: `Module` 类的 `forward` 函数里多次调用同一个层。此外，如果我们传入 `Sequential` 的模块是同一个 `Module` 实例的话参数也是共享的，下面来看一个例子:

In [14]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)  # 同一个实例串联
print(net)

for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


在内存中，这两个线性层其实一个对象:

In [15]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:

$y = (x \times w) \times w, w=3 $

In [17]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)

y.backward()
print(net[0].weight.grad) # 单次梯度是 3，两次所以就是 6

tensor(9., grad_fn=<SumBackward0>)
tensor([[12.]])


### 小结

- 有多种方法来访问、初始化和共享模型参数。
- 可以自定义初始化方法。

--- 

## 自定义层

>(Restart your kernel here)

深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的卷积层、池化层与循环层。虽然 PyTorch 提供了大量常用的层，但有时候我们依然希望自定义层。本节将介绍如何使用 `Module` 来自定义层，从而可以被重复调用。

### 不含模型参数的自定义层

我们先介绍如何定义一个不含模型参数的自定义层。事实上，这和上面「模型构造」中介绍的使用 `Module` 类构造模型类似。下面的 `CenteredLayer` 类通过继承 `Module` 类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了 `forward` 函数里。这个层里不含模型参数。

In [1]:
import torch
from torch import nn

class CenteredLayer(nn.Module):   # 不含模型参数
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

我们可以实例化这个层，然后做前向计算。

In [2]:
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

我们也可以用它来构造更复杂的模型。

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

下面打印自定义层各个输出的均值。因为均值是浮点数，所以它的值是一个很接近 0 的数。

In [4]:
y = net(torch.rand(4, 8))
y.mean().item()

-3.725290298461914e-09

### 含模型参数的自定义层

我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。

在上面「模型参数的访问、初始化和共享」中介绍了 `Parameter` 类其实是 `Tensor` 的子类，如果一个 `Tensor` 是 `Parameter`，那么它会自动被添加到模型的参数列表里。所以在自定义含模型参数的层时，我们应该将参数定义成 `Parameter`，除了像「模型参数的访问、初始化和共享」节那样直接定义成 `Parameter` 类外，还可以使用 `ParameterList` 和 `ParameterDict` 分别定义参数的列表和字典。

- `ParameterList` 接收一个 `Parameter` 实例的列表作为输入然后得到一个参数列表，使用的时候可以用索引来访问某个参数，另外也可以使用 `append` 和 `extend` 在列表后面新增参数。

In [5]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
net = MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


- `ParameterDict` 接收一个 `Parameter` 实例的字典作为输入然后得到一个参数字典，然后可以按照字典的规则使用了。例如使用 `update()` 新增参数，使用 `keys()` 返回所有键值，使用 `items()` 返回所有键值对等等，可参考[官方文档](https://pytorch.org/docs/stable/nn.html#parameterdict)。

In [6]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


这样就可以根据传入的键值来进行不同的前向传播：

In [7]:
x = torch.ones(1, 4)
print(net(x, 'linear1'))
print(net(x, 'linear2'))
print(net(x, 'linear3'))

tensor([[ 2.4429, -2.9363, -2.8187, -0.3683]], grad_fn=<MmBackward>)
tensor([[-1.2958]], grad_fn=<MmBackward>)
tensor([[-0.3718, -0.7315]], grad_fn=<MmBackward>)


我们也可以使用自定义层构造模型。它和 PyTorch 的其他层在使用上很类似。

In [8]:
net = nn.Sequential(
    MyDictDense(),
    MyListDense(),
)
print(net)
print(net(x))

Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): MyListDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)
tensor([[-23.8090]], grad_fn=<MmBackward>)


### 小结

- 可以通过 `Module` 类自定义神经网络中的层，从而可以被重复调用。

---

## 读取和存储

>(Restart your kernel here)

到目前为止，我们介绍了如何处理数据以及如何构建、训练和测试深度学习模型。然而在实际中，我们有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使用。

### 读写Tensor

我们可以直接使用 `save` 函数和 `load` 函数分别存储和读取 `Tensor`。`save` 使用 Python 的 pickle 实用程序将对象进行序列化，然后将序列化的对象保存到 disk，使用 `save` 可以保存各种对象，包括模型、张量和字典等。而 `load` 使用 unpickle 工具将 pickle 的对象文件反序列化为内存。

- 下面的例子创建了 `Tensor` 变量 `x`，并将其存在文件名同为 `x.pt` 的文件里。

In [1]:
import torch
from torch import nn

x = torch.ones(3)
torch.save(x, 'x.pt')

In [2]:
!ls -lh x.pt # 本地目录

-rw-r--r--  1 herb  staff   351B  3 27 10:12 x.pt


然后我们将数据从存储的文件读回内存。

In [3]:
x2 = torch.load('x.pt')
x2

tensor([1., 1., 1.])

- 我们还可以存储一个 `Tensor` 列表并读回内存：

In [4]:
y = torch.zeros(4)
torch.save([x, y], 'xy.pt')
xy_list = torch.load('xy.pt')
xy_list

[tensor([1., 1., 1.]), tensor([0., 0., 0., 0.])]

- 存储并读取一个从字符串映射到 `Tensor` 的字典：

In [5]:
torch.save({'x': x, 'y': y}, 'xy_dict.pt')
xy = torch.load('xy_dict.pt')
xy

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

### 读写模型

#### state_dict

在 PyTorch 中，`Module` 的可学习参数(即权重和偏差)，模块模型包含在参数中(通过 `model.parameters()` 访问)。`state_dict` 是一个从参数名称隐射到参数 `Tesnor` 的字典对象。

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(3, 2)
        self.act = nn.ReLU()
        self.output = nn.Linear(2, 1)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

net = MLP()
net.state_dict()

OrderedDict([('hidden.weight', tensor([[ 0.4776, -0.0531,  0.5554],
                      [ 0.0822, -0.4127, -0.5208]])),
             ('hidden.bias', tensor([-0.3275,  0.1323])),
             ('output.weight', tensor([[ 0.4475, -0.0775]])),
             ('output.bias', tensor([-0.3367]))])

>注意，只有具有可学习参数的层(卷积层、线性层等)才有 `state_dict` 中的条目。优化器(optim)也有一个 `state_dict`，其中包含关于优化器状态以及所使用的超参数的信息。

In [7]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [140575174809928,
    140575174810000,
    140575174810072,
    140575174810144]}]}

### 保存和加载模型

PyTorch中保存和加载训练模型有两种常见的方法:

- 仅保存和加载模型参数 (`state_dict`)；
- 保存和加载整个模型。

1. 保存和加载模型参数 `state_dict`（**推荐方式**）
>由于保存整个模型将耗费大量的存储，故官方推荐只保存参数，然后在建好模型的基础上加载。

    保存：
    
```python
>>torch.save(model.state_dict(), PATH) # 推荐的文件后缀名是pt或pth
```

    加载：
    
```python
>>model = TheModelClass(*args, **kwargs);
>>model.load_state_dict(torch.load(PATH));
```
    
2. 保存和加载整个模型

    保存：

```python
>>torch.save(model, PATH)
```

    加载：
    
```python
>>model = torch.load(PATH)
```

我们采用推荐的方法一来实验一下:

In [8]:
X = torch.randn(2, 3)
Y = net(X)

PATH = "./net.pt"
torch.save(net.state_dict(), PATH)

net2 = MLP()
net2.load_state_dict(torch.load(PATH))
Y2 = net2(X)
Y2 == Y

tensor([[True],
        [True]])

因为这 `net` 和 `net2` 都有同样的模型参数，那么对同一个输入 `X` 的计算结果将会是一样的。上面的输出也验证了这一点。

A common PyTorch convention is to save models using either a `.pt` or `.pth` file extension.

此外，还有一些其他使用场景，例如 GPU 与 CPU 之间的模型保存与读取、使用多块 GPU 的模型的存储等等，使用的时候可以参考[官方文档](https://pytorch.org/tutorials/beginner/saving_loading_models.html)。

### 小结

- 通过 `save` 函数和 `load` 函数可以很方便地读写 `Tensor`。
- 通过 `save` 函数和 `load_state_dict` 函数可以很方便地读写模型的参数。

---

## GPU计算

>(Restart your kernel here)

到目前为止，我们一直在使用 CPU 计算。对复杂的神经网络和大规模的数据来说，使用 CPU 来计算可能不够高效。在本节中，我们将介绍如何使用单块 NVIDIA GPU 来计算。所以需要确保已经安装好了 PyTorch GPU 版本。准备工作都完成后，下面就可以通过 `nvidia-smi` 命令来查看显卡信息了。

In [1]:
!nvidia-smi  # 对Linux/macOS用户有效

Mon Mar 22 09:34:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.54                 Driver Version: 396.54                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:03:00.0  On |                  N/A |
| 21%   35C    P8    18W / 250W |    951MiB / 11177MiB |     37%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   39C    P8    18W / 250W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

```text
Mon Mar 22 09:05:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.54                 Driver Version: 396.54                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:03:00.0  On |                  N/A |
| 21%   35C    P5    23W / 250W |    951MiB / 11177MiB |      8%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 21%   37C    P8    18W / 250W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  GeForce GTX 108...  Off  | 00000000:81:00.0 Off |                  N/A |
| 21%   34C    P8    16W / 250W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   3  GeForce GTX 108...  Off  | 00000000:82:00.0 Off |                  N/A |
| 21%   33C    P8    16W / 250W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
+-----------------------------------------------------------------------------+

```

可以看到我这里只有四块 GTX 1080Ti，每块显存一共有 12000 M。

### 计算设备

PyTorch 可以指定用来存储和计算的设备，如使用内存的 CPU 或者使用显存的 GPU。默认情况下，PyTorch 会将数据创建在内存，然后利用 CPU 来计算。

用 `torch.cuda.is_available()` 查看 GPU 是否可用:

In [2]:
import torch
from torch import nn

torch.cuda.is_available() # 输出 True 为 GPU 可用

True

查看 GPU 数量：

In [3]:
torch.cuda.device_count() # 输出 4

4

查看当前 GPU 索引号，索引号从 0 开始：

In [4]:
torch.cuda.current_device() # 输出 0

0

根据索引号查看 GPU 名字:

In [5]:
torch.cuda.get_device_name(0) # 输出 'GeForce GTX 1080 Ti'

'GeForce GTX 1080 Ti'

### `Tensor` 的 GPU 计算

默认情况下，`Tensor` 会被存在内存上。因此，之前我们每次打印 `Tensor` 的时候看不到 GPU 相关标识。

In [6]:
x = torch.tensor([1, 2, 3])
x

tensor([1, 2, 3])

使用 `.cuda()` 可以将 CPU 上的 `Tensor` 转换（复制）到 GPU 上。如果有多块 GPU，我们用 `.cuda(i)` 来表示第 i 块 GPU 及相应的显存（i从0开始）且 `cuda(0)` 和 `cuda()` 等价。

In [7]:
x = x.cuda(0)
x

tensor([1, 2, 3], device='cuda:0')

我们可以通过 `Tensor` 的 `device` 属性来查看该 `Tensor` 所在的设备。

In [8]:
x.device

device(type='cuda', index=0)

我们可以直接在创建的时候就指定设备，使用 `torch.device`。

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x = torch.tensor([1, 2, 3], device=device)
# or
x = torch.tensor([1, 2, 3]).to(device)
x

tensor([1, 2, 3], device='cuda:0')

如果对在 GPU 上的数据进行运算，那么结果还是存放在 GPU 上。

In [10]:
y = x**2
y

tensor([1, 4, 9], device='cuda:0')

需要注意的是，存储在不同位置中的数据是不可以直接进行计算的。即存放在 CPU 上的数据不可以直接与存放在 GPU 上的数据进行运算，位于不同 GPU 上的数据也是不能直接进行计算的。

In [11]:
y + x.cpu() # 会报错

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [12]:
y = y.to("cuda:1")
y + x  # 也会报错

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:0!


![](https://vip2.loli.io/2021/03/22/tTvcxoVbz5D876w.png)


In [13]:
z = x.to("cuda:1")
z + y

tensor([ 2,  6, 12], device='cuda:1')

### 模型的 GPU 计算

同 `Tensor` 类似，PyTorch 模型也可以通过 `.cuda` 转换到GPU上。我们可以通过检查模型的参数的 `device` 属性来查看存放模型的设备。

In [14]:
net = nn.Linear(3, 1)
list(net.parameters())[0].device 

device(type='cpu')

可见模型在 CPU 上，将其转换到 GPU 上:

In [15]:
net.cuda() # Moves all model parameters and buffers to the GPU.
list(net.parameters())[0].device

device(type='cuda', index=0)

同样的，我么需要保证模型输入的 `Tensor` 和模型都在同一设备上，否则会报错。

In [16]:
x = torch.rand(2,3).cuda()
net(x)

tensor([[-0.6742],
        [-0.1176]], device='cuda:0', grad_fn=<AddmmBackward>)

### 小结

- PyTorch 可以指定用来存储和计算的设备，如使用内存的 CPU 或者使用显存的 GPU。在默认情况下，PyTorch 会将数据创建在内存，然后利用 CPU 来计算。
- PyTorch 要求计算的所有输入数据都在内存或同一块显卡的显存上。